# 一、加载常用库

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import SimpleITK as sitk
from tqdm import tqdm

%matplotlib inline
from glob import glob
import warnings #不显示乱七八糟的warning
warnings.filterwarnings("ignore")

In [2]:
from solo import *
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


ImportError: No module named hyperopt

## 二、数据获取

In [ ]:
luna_path = './'
luna_subset_path = luna_path + 'sample_patients/'
file_list = glob(luna_subset_path + "*.mhd")
df_node = pd.read_csv(luna_path + 'csv_files/' + 'annotations.csv')

output_path = luna_path + 'npy/'
working_path = luna_path + 'output/'

In [ ]:
df_node["file"] = df_node["seriesuid"].map(lambda file_name: get_filename(file_list, file_name))
df_node = df_node.dropna()
df_node.head(5)

In [ ]:
for fcount, img_file in enumerate(tqdm(file_list)):
    mini_df = df_node[df_node["file"]==img_file] #get all nodules associate with file
    if mini_df.shape[0]>0: # some files may not have a nodule--skipping those 
        # load the data once
        ct_scan, origin, spacing = load_itk(img_file)
        for node_idx, cur_row in mini_df.iterrows():       
            node_x = cur_row["coordX"]
            node_y = cur_row["coordY"]
            node_z = cur_row["coordZ"]
            diam = cur_row["diameter_mm"]
            cands = np.array([0,node_x, node_y, node_z, diam])
            center = np.array([node_x, node_y, node_z])
            voxel_coordinates = world_2_voxel(center, origin, spacing)

In [ ]:
plt.imshow(ct_scan[0])

In [ ]:
mini_df2 = mini_df.copy()
del mini_df2['file']
mini_df2

In [ ]:
image_mask = draw_circles(ct_scan[0], mini_df2, origin, spacing)